# Clasificación de Imágenes MRI con RNN

Este notebook explora el uso de redes neuronales recurrentes (RNN) para la clasificación de imágenes MRI de tumores cerebrales. Incluye:

1. Preprocesamiento y conversión de imágenes a secuencias
2. Definición y entrenamiento de un modelo RNN
3. Evaluación de resultados

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense, Flatten

# Configuración de rutas
DATASET_DIR = "../total/archive/Training"

In [ ]:
# Ejemplo de modelo RNN para imágenes (requiere convertir imágenes a secuencias)
# model = Sequential([
#     SimpleRNN(128, input_shape=(secuencia_largo, secuencia_dim)),
#     Dense(num_classes, activation='softmax')
# ])
# model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

Las RNN no son ideales para imágenes, pero este notebook es educativo para explorar su uso en visión computacional.

# Clasificación de Tumores Cerebrales con RNN (Secuencias de Parches de Imagen)

Este notebook implementa una aproximación experimental para usar RNN en imágenes MRI, convirtiendo cada imagen en una secuencia de parches y clasificando con una red recurrente.

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense, Dropout, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam

# Parámetros
IMG_SIZE = 128  # Tamaño reducido para secuencias
PATCH_SIZE = 16
SEQUENCE_LENGTH = (IMG_SIZE // PATCH_SIZE) ** 2
PATCH_DIM = PATCH_SIZE * PATCH_SIZE * 3
BATCH_SIZE = 32
EPOCHS = 50
LEARNING_RATE = 0.001

BASE_DIR = Path(os.path.join(os.getcwd(), '../total/archive/Training'))

# Cargar datos y convertir imágenes a secuencias de parches
# ...

In [ ]:
def image_to_sequence(img, patch_size=PATCH_SIZE):
    patches = []
    h, w, c = img.shape
    for i in range(0, h, patch_size):
        for j in range(0, w, patch_size):
            patch = img[i:i+patch_size, j:j+patch_size, :].reshape(-1)
            patches.append(patch)
    return np.array(patches)

# Ejemplo de conversión de una imagen
# img = ...
# seq = image_to_sequence(img)

In [ ]:
# Definir modelo RNN
model = Sequential([
    SimpleRNN(128, input_shape=(SEQUENCE_LENGTH, PATCH_DIM)),
    BatchNormalization(),
    Dropout(0.5),
    Dense(64, activation='relu'),
    BatchNormalization(),
    Dropout(0.3),
    Dense(4, activation='softmax')  # Cambia 4 por el número de clases
])
model.compile(optimizer=Adam(learning_rate=LEARNING_RATE), loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

**Nota:** Este enfoque es experimental y no es óptimo para imágenes, pero puede ser útil para explorar el uso de RNN en visión computacional.